# Solar Abundance Distribution with OMEGA

This notebook shows how to plot the predicted abundance distribution of the galactic gas at a certain time, and to quantify the contribution of different enrichment sources (e.g., massive stars, Type Ia supernovae, etc.).  This uses the galactic chemical evolution code OMEGA.

In [ ]:
# Load modules
import matplotlib
import matplotlib.pyplot as plt
import sygma
#reload(sygma)
import omega
#reload(omega)

## 1. Run an OMEGA instance and isolate where the Sun forms

In [ ]:
# Run a simple Milky Way simulation with no outflows.
# Uses NuGrid yields (Ritter et al. 2017)
o_sol = omega.omega(galaxy='milky_way', special_timesteps=100, DM_evolution=True, iniZ=0.0001, \
      table='yield_tables/agb_and_massive_stars_nugrid_MESAonly_fryer12delay.txt', \
        mass_frac_SSP=0.35, exp_ml=1.0, nb_1a_per_m=1.5e-3, sfe=0.05, t_sf_z_dep=0.3, mass_loading=0.0, \
          transitionmass=10)

In [ ]:
# Find the timestep index where the gas metallicity is ~0.014 (solar value).
i_step_sol = 87
print o_sol.history.metallicity[i_step_sol]

## 2. Prepare isotopes and elements lists, and read solar abundances

In [ ]:
# Function to retrun the list of isotope index for an input element
def get_list_iso_index(specie, inst):
    specie_list = []
    for i_gl in range(0,len(inst.history.isotopes)):
        if (specie+'-') in inst.history.isotopes[i_gl]:
            specie_list.append(i_gl)
    return specie_list

# List of charge numbers (NEED to be as in the yields table)
Z_charge = []
for i in range(1,84):
    # Exclude elements not considered in NuGrid yields
    if (not i == 43) and (not i == 61):
        Z_charge.append(i)
        
# List of elements (NuGrid yields)
elements = ['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', 'Mg', 'Al', \
            'Si', 'P', 'S', 'Cl', 'Ar', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', \
            'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', \
            'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', \
            'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Sm', \
            'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', \
            'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi']

# List of atomic weigth to plot the mass fractions
atom_weigth = [1.0079, 4.0026, 6.941, 9.0122, 10.81, 12.011, 14.007, 15.999, 18.998, 20.180, \
               22.990, 24.305, 26.982, 28.086, 30.974, 32.065, 35.453, 39.453, 39.098, \
               40.078, 44.956, 47.867, 50.942, 51.996, 54.938, 55.845, 58.933, 58.693, \
               63.546, 65.390, 69.723, 72.610, 74.922, 78.96, 79.904, 83.80, 85.468, \
               87.62, 88.906, 91.224, 92.906, 95.94, 98.0, 101.07, 102.91, 106.42, 107.87, \
               112.41, 114.82, 118.71, 121.76, 127.60, 126.90, 131.29, 132.91, 137.33, \
               138.91, 140.12, 140.91, 144.24, 145.0, 150.36, 151.96, 157.25, 158.93, 162.50, \
               164.93, 167.26, 168.93, 173.04, 174.97, 178.49, 180.95, 183.84, 186.21, \
               190.23, 192.22, 195.08, 196.97, 200.59, 204.38, 207.2, 208.98]

In [ ]:
# Read solar abundances (number densities)
solar_Z = []
solar_ab = []
solar_ab_path = 'Lodders_et_al_2009.txt'
with open("/opt/NuPyCEE/stellab_data/solar_normalization/"+\
           solar_ab_path, 'r') as f:
    not_finished = True
    for line in f:
        split_line = [str(x) for x in line.split()]
        if not_finished:
            solar_Z.append(int(split_line[0]))
            solar_ab.append(10**(float(split_line[2])-12))
            if split_line[1] == 'Bi':
                not_finished = False
f.close()

# Convert number of atoms into masses
for i in range(len(solar_ab)):
    solar_ab[i] *= atom_weigth[i]
    
# Normalize to 1.0
norm = 1.0/sum(solar_ab)
for i in range(len(solar_ab)):
    solar_ab[i] *= norm

## 3. Calculate the contribution of different sources

In [ ]:
# Function to extract the contribution of individual sources
def get_individual_sources(inst, iRAWDs_on, i_step_sol):
    
    # Declare the abundances arrays
    m_el_all = []
    m_el_agb = []
    m_el_massive = []
    m_el_sn1a = []
    m_el_nsm = []
    if iRAWDs_on:
        m_el_iRAWDs = []
    
    # Get the mass distrubution of individual sources
    for i_el in range(0,len(elements)):
        m_el_all.append(0.0)
        m_el_agb.append(0.0)
        m_el_massive.append(0.0)
        m_el_sn1a.append(0.0)
        m_el_nsm.append(0.0)
        if iRAWDs_on:
            m_el_iRAWDs.append(0.0)
        specie_list = get_list_iso_index(elements[i_el], inst)
        for i_iso in range(0,len(specie_list)):
            m_el_all[i_el] += inst.ymgal[i_step_sol][specie_list[i_iso]]
            m_el_agb[i_el] += inst.ymgal_agb[i_step_sol][specie_list[i_iso]]
            m_el_massive[i_el] += inst.ymgal_massive[i_step_sol][specie_list[i_iso]]
            m_el_sn1a[i_el] += inst.ymgal_1a[i_step_sol][specie_list[i_iso]]
            m_el_nsm[i_el] += inst.ymgal_nsm[i_step_sol][specie_list[i_iso]]
            if iRAWDs_on:
                m_el_iRAWDs[i_el] += inst.ymgal_delayed_extra[0][i_step_sol][specie_list[i_iso]]
    
    # Normalize each sources
    norm_all_for_all = 1.0 / sum(m_el_all)
    for i_el in range(0,len(elements)):
        m_el_all[i_el] *= norm_all_for_all
        m_el_agb[i_el] *= norm_all_for_all
        m_el_massive[i_el] *= norm_all_for_all
        m_el_sn1a[i_el] *= norm_all_for_all
        m_el_nsm[i_el] *= norm_all_for_all
        if iRAWDs_on:
            m_el_iRAWDs[i_el] *= norm_all_for_all
    
    # Return abundances patterns
    if iRAWDs_on:
        return m_el_all, m_el_agb, m_el_massive, m_el_sn1a, m_el_nsm, m_el_iRAWDs
    else:
        return m_el_all, m_el_agb, m_el_massive, m_el_sn1a, m_el_nsm

In [ ]:
# Get contribution
m_el_all, m_el_agb, m_el_massive, m_el_sn1a, m_el_nsm = \
    get_individual_sources(o_sol, iRAWDs_on=False, i_step_sol=i_step_sol)

## 4. Plot abundance pattern

In [ ]:
# Define the figure size
fig = plt.figure(figsize=(6,4.))
matplotlib.rcParams.update({'font.size': 14.0})

# Plot solar abundance data
plt.plot(solar_Z, solar_ab, color='c', linewidth=4, alpha=0.5, label='Solar')

# Plot predicted contribution 
plt.plot(Z_charge, m_el_all, color='orange', label='All sources', alpha=1.0, linestyle='-', linewidth=2)
plt.plot(Z_charge, m_el_sn1a, color='r', label='SNe Ia', alpha=0.8, linestyle='--', marker='s')
plt.plot(Z_charge, m_el_massive, color='b', label='Massive', alpha=0.8, linestyle='--', marker='^')
plt.plot(Z_charge, m_el_agb, color='g', label='AGB', alpha=0.8, linestyle='-', marker='x')

# Annotation of the different elements
ft = 14.0
plt.annotate('Ti',xy=(22,8e-6),   fontsize=ft, ha='center',va='bottom')
plt.annotate('V', xy=(23,3e-9),   fontsize=ft, ha='center',va='bottom')
plt.annotate('Cr',xy=(24,5e-4),   fontsize=ft, ha='center',va='bottom')
plt.annotate('Mn',xy=(25,1.3e-7), fontsize=ft, ha='center',va='bottom')
plt.annotate('Fe',xy=(26,3e-3),   fontsize=ft, ha='center',va='bottom')
plt.annotate('Co',xy=(27,5e-8),   fontsize=ft, ha='center',va='bottom')
plt.annotate('Ni',xy=(28,4e-4),   fontsize=ft, ha='center',va='bottom')
plt.annotate('Cu',xy=(29,1e-9),   fontsize=ft, ha='center',va='bottom')
plt.annotate('Zn',xy=(30,1e-5),   fontsize=ft, ha='center',va='bottom')
plt.annotate('Ga',xy=(31,1.3e-9), fontsize=ft, ha='center',va='bottom')

# Label and axis
plt.legend(fontsize=13, frameon=False)
plt.xlabel('Z (charge number)', fontsize=16)
plt.ylabel('X (mass fraction)', fontsize=16)
plt.xlim(20,34)
plt.ylim(1e-10,3e-1)
plt.yscale('log')

# Frame tuning
plt.subplots_adjust(top=0.95)
plt.subplots_adjust(right=0.98)
plt.subplots_adjust(left=0.15)
plt.subplots_adjust(bottom=0.14)

## 5. Investigate the origin of a particular element

In [ ]:
# Run SYGMA (simple stellar population instances)
s_0_02  = sygma.sygma(iniZ=0.02)
s_0_01  = sygma.sygma(iniZ=0.01)
s_0_006 = sygma.sygma(iniZ=0.006)

In [ ]:
# Select the element
specie = 'Cr'

# Print a warning
print 'Warning! This plot does not show the contribution of Type Ia supernovae.'

# Plot the yields weighted by the initial mass function
s_0_02.plot_mass_range_contributions(specie=specie,  log=True, color='b', label='Z=0.02')
s_0_01.plot_mass_range_contributions(specie=specie,  log=True, color='g', label='Z=0.01')
s_0_006.plot_mass_range_contributions(specie=specie, log=True, color='r', label='Z=0.06')

# Rearange the figure (legend, axis labels)
plt.figsize=(6,4.4)
plt.legend(loc=2, frameon=False, fontsize=13)
plt.xlim(1,27)
plt.ylabel('Cr IMF-weighted yields [M$_\odot$]')
plt.xlabel('Stellar initial mass [M$_\odot$]')